In [3]:
import pandas as pd

awards_players_df = pd.read_csv("data/awards_players.csv")
coaches_df = pd.read_csv("data/coaches.csv")
players_teams_df = pd.read_csv("data/players_teams.csv")
players_df = pd.read_csv("data/players.csv")
series_post_df = pd.read_csv("data/series_post.csv")
teams_post_df = pd.read_csv("data/teams_post.csv")
teams_df = pd.read_csv("data/teams.csv")
teams_df.isna().any()

year        False
lgID        False
tmID        False
franchID    False
confID      False
            ...  
confW       False
confL       False
min         False
attend      False
arena       False
Length: 61, dtype: bool